In [1]:
import os
%pwd

'/export/home/ingole/My Files/Projects/Text-Synthesis-and-Compression-Project/Text-Synthesis-and-Compression-Project/reserach'

In [2]:
os.chdir("../")

In [3]:
%pwd

'/export/home/ingole/My Files/Projects/Text-Synthesis-and-Compression-Project/Text-Synthesis-and-Compression-Project'

In [4]:
#Defining the entity here

from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataTransformationConfig:
    root_dir: Path
    data_path: Path
    tokenizer_name: Path

In [5]:
from src.textSynthesisAndCompression.constants import *
from src.textSynthesisAndCompression.utils.common import read_yaml, create_directories

In [6]:
class ConfigurationManager:
    def __init__(self,
                config_filepath = CONFIG_FILE_PATH,
                params_filepath = PARAMS_FILE_PATH):
        
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        
        create_directories([self.config.artifacts_root])
        
    def get_data_transformation_config(self) -> DataTransformationConfig:
        config = self.config.data_transformation
        
        create_directories([config.root_dir])
        
        data_transformation_config = DataTransformationConfig(
            root_dir = config.root_dir,
            data_path = config.data_path,
            tokenizer_name = config.tokenizer_name
        )
        
        return data_transformation_config

In [7]:
import os
from src.textSynthesisAndCompression.logging import logger
from transformers import AutoTokenizer
from datasets import load_dataset, load_from_disk

[2024-01-08 02:55:54,138: INFO: config: PyTorch version 2.0.1 available.]
[2024-01-08 02:55:54,142: INFO: config: TensorFlow version 2.12.0 available.]


In [12]:
class DataTransformation:
    def __init__(self, config: DataTransformationConfig):
        self.config = config
        self.tokenizer = AutoTokenizer.from_pretrained(config.tokenizer_name)
        
    def convert_examples_to_features(self, example_batch):
        input_encodings = self.tokenizer(example_batch['dialogue'], max_length=1024, truncation=True)
        
        with self.tokenizer.as_target_tokenizer():
            target_encodings = self.tokenizer(example_batch['summary'], max_length=1024, truncation=True)
            
        return {
            'input_ids': input_encodings['input_ids'],
            'attention_mask': input_encodings['attention_mask'],
            'labels': target_encodings['input_ids']
        }
    
    def convert(self):
        dataset_samsum = load_from_disk(self.config.data_path)
        dataset_samsum_pt = dataset_samsum.map(self.convert_example_to_features, batcher=True)
        dataset_samsum_pt.save_to_disk(os.path.join(self.config.root_dir,"samsum_dataset"))

In [18]:
try:
    config = ConfigurationManager()
    data_transformation_config = config.get_data_transformation_config()
    data_transformation = DataTransformation(config=data_transformation_config)
    data_transformation.convert()
except Exception as e:
    raise e

[2024-01-08 18:35:02,520: INFO: common: yaml file: config/config.yaml loaded successfully]
[2024-01-08 18:35:02,523: INFO: common: yaml file: params.yaml loaded successfully]
[2024-01-08 18:35:02,525: INFO: common: Created directory at: artifacts]
[2024-01-08 18:35:02,528: INFO: common: Created directory at: artifacts/data_transformation]


FileNotFoundError: Directory artifacts/data_ingestion/train.json is neither a `Dataset` directory nor a `DatasetDict` directory.